# LIBRARIES

In [7]:
import os
import faiss
import numpy as np
import joblib
import re

from pypdf import PdfReader
from sentence_transformers import SentenceTransformer

# RAG DOCUMENTS

In [8]:
BASE_PATH = "models"
EMBED_PATH = os.path.join(BASE_PATH, "embedding_model")

os.makedirs(EMBED_PATH, exist_ok=True)

print("Storage folders ready.")

Storage folders ready.


In [9]:
PDF_PATH = "rag/telecom_doc.pdf"   


def load_pdf(file_path):

    reader = PdfReader(file_path)

    text = ""

    for page in reader.pages:

        page_text = page.extract_text()

        if page_text:
            text += page_text + "\n"

    return text


text_data = load_pdf(PDF_PATH)

print("Document loaded.")
print("Characters:", len(text_data))

Document loaded.
Characters: 9832


In [10]:
def chunk_text_smart(text,
                     chunk_size=300,
                     overlap=50):

    # Split into sentences
    sentences = re.split(r'(?<=[.!?]) +', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:

        # If adding sentence stays within limit
        if len(current_chunk) + len(sentence) <= chunk_size:

            current_chunk += " " + sentence

        else:

            # Save chunk
            chunks.append(current_chunk.strip())

            # Start new chunk with overlap
            overlap_text = current_chunk[-overlap:]

            current_chunk = overlap_text + " " + sentence

    # Add last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [14]:
chunks = chunk_text_smart(text_data,
                          chunk_size=300,
                          overlap=50)

print("Total chunks:", len(chunks))

Total chunks: 36


# EMBEDDING MODEL 

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

embedding_model.save(EMBED_PATH)

print("Embedding model saved.")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 666.07it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  9.90it/s]

Embedding model saved offline.


In [15]:
embeddings = embedding_model.encode(chunks)

embeddings = np.array(embeddings).astype("float32")

print("Embeddings shape:", embeddings.shape)

Embeddings shape: (36, 384)


In [16]:
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)

index.add(embeddings)

print("FAISS index created.")

FAISS index created.


In [ ]:
rag_artifacts = {
    "faiss_index": index,
    "chunks": chunks
}

joblib.dump(rag_artifacts, "models/rag_artifacts.pkl")

print("RAG model saved ")

RAG model saved using joblib.


# RAG EVALUATION

In [18]:
def retrieve_context(query, top_k=3):

    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")

    distances, indices = index.search(query_embedding, top_k)

    retrieved_chunks = [chunks[i] for i in indices[0]]

    context = "\n".join(retrieved_chunks)

    return context

In [19]:
print(retrieve_context("What is the refund policy?"))

pany Policies
Billing: Monthly billing in advance. Enterprise customers receive consolidated invoices. Late payment after 7
days may suspend services.
Refund:  Full  refund  within  7  days  if  usage  is  less  than  10%.
within  7  days  if  usage  is  less  than  10%. Cloud  services  are  non-refundable  after
activation.
3
Contracts: Individual users have no long-term lock-in.
racts: Individual users have no long-term lock-in. Enterprise customers have a minimum 12-month
contract.
SLA: Individual users have 98.5% uptime, business users 99.5% uptime, and enterprise users 99.9% uptime.
Data Privacy: GDPR compliant, ISO 27001 certified, and encrypted data at rest and in transit.
Fair Usage: Unlimited plans are capped at 1TB per month.
Support Tiers: Standard, Priority, and Enterprise Dedicated Support.
Discounts: Bulk enterprise users can receive up to 30% discount.
